In [13]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from graspy.simulations import binary_sbm
from graspy.utils import symmetrize
from tqdm import tqdm_notebook as tqdm
import warnings
warnings.filterwarnings("ignore")

In [73]:
def rho_sbm(rho,k,L,n=100):
    """
    Generates 2 adjacency matrices A,B of graphs G1, G2.
    (G1,G2) are sampled from a rho-SBM(k,b,L) by the method described in [1].
    The block membership function assumes the first n//k belong to block 1, and so on.
    
    Parameters
    ----------
    rho : float, int
        correlation between graphs.
        0.0 <= rho <= 1.0
    k : int
        number of blocks of the rho-SBM.
        k >= 1
    L : array-like
        edge probability matrix, each entry a float between 0 and 1.
        L should be of size k by k.
    n : int, optional, default = 100
        number of vertices for each graph
    
    Returns
    -------
    A : array-like
        The adjacency matrix of graph G1
    B : array-like
        The adjacency matrix of graph G2

    References
    ----------
    .. [1] Fishkind et. al (2012).
        Seeded Graph Matching
        arXiv:1209.0367v4
    """
    L = symmetrize(L)
    A = binary_sbm([int(n/k)]*k, L)
    BL = np.repeat(np.repeat(L,n//k,0),n//k,1)
    B = np.random.binomial(1,(1-rho)*BL + rho*A)
    np.fill_diagonal(B,0)
    return A,B

def rho_h(nvec,pvec):
    mu = np.sum(np.multiply(nvec,pvec))/np.sum(nvec)
    var1 = np.square(np.subtract(pvec,mu))
    var = np.sum(np.multiply(nvec,var1))/np.sum(nvec)
    return var/(mu*(1-mu))

def corr(A,B):
    A = A.ravel()
    B = B.ravel()
    x = np.vstack((A.ravel(),B.ravel()))
    return np.corrcoef(x)[0,1]

In [46]:
def gen_ER_data(MONTE_ITER=100):
    allL = []
    for L in tqdm(np.linspace(0,1,11)):
        someL = []
        for rho in np.linspace(0,1,101):
            monte = []
            for _ in range(MONTE_ITER):
                A,B = rho_sbm(rho, 1, np.array([[L]]))
                c = corr(A,B)
                monte.append(c)
            y = np.mean(monte)
            yerr = np.std(monte)
            someL.append((y,yerr))
        allL.append(someL)
    return np.array(allL)

In [48]:
x = np.linspace(0,1,11)
y = np.linspace(0,1,101)
z = gen_ER_data()

In [70]:
def rho_h_t(rho_e, rho_h):
    return 1-(1-rho_e)*(1-rho_h)

def rho_t(nvec,pvec,rho_e):
    h = rho_h(nvec,pvec)
    return rho_h_t(rho_e,h)

rhoh = []
for i in tqdm(y):
    rhoh.append(rho_t([int(100/1)]*1, np.diag(np.array([L])), i))

In [57]:
import plotly.plotly as py
import plotly.graph_objs as go

data = [
    go.Surface(
        x=y,
        y=x,
        z=z[:,:,0]
    ),
    go.Surface(
        x=y,
        y=x,
        z=z[:,:,0]+z[:,:,1],
        showscale=False, opacity=0.8
    ),
    go.Surface(
        x=y,
        y=x,
        z=z[:,:,0]-z[:,:,1],
        showscale=False, opacity=0.8
    ),
]

layout = go.Layout(
    title='rho-ER',
    autosize=False,
    scene = dict(
        xaxis = dict(
            title='rho parameter'),
        yaxis = dict(
            title='edge probability'),
        zaxis = dict(
            title='calculated rho') 
    ),
    width=750,
    height=750,
    margin=dict(
        l=65,
        r=50,
        b=65,
        t=90
    )
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

In [ ]:
plt.errorbar(np.linspace(0,1,101),y,yerr=yerr)
plt.title('rho of of rho-sbms (100 monte)')
plt.xlabel(r"$\rho$ input")
plt.ylabel(r'$\rho$ output')
plt.show()

In [47]:
def gen_z(L,rho,MONTE_ITER=100):
    monte = []
    for _ in range(MONTE_ITER):
        A,B = rho_sbm(rho, 1, np.array([[L]]))
        c = corr(A,B)
        monte.append(c)
    y = np.mean(monte)
    yerr = np.std(monte)